Challenge: ccv1  
# Deep Learning: Was versteckt sich da?  
## Demo Modell
In diesem Notebook können Bilder mit unterschiedlichen Megadetector Threshold mit dem Modell getestet werden

In [ ]:
import os
import ast
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torchvision.transforms as transforms
from PIL import Image, ImageEnhance

 ## Test Bild
 Bilder werden zufällig aus dem `demo_images` Ordner geladen. (ggf. random.seed() kommentieren für verschiedene Bilder)

In [ ]:
random.seed(14)

folder_path = './demo_images'
images = os.listdir(folder_path)

random_image_name = random.choice(images)
random_image_path = os.path.join(folder_path, random_image_name)
random_image = Image.open(random_image_path)
display(random_image_name[:-4])
display(random_image)

## Demo Modell

In [ ]:
# load modell

# predict animal class